In [8]:
import pandas as pd

In [27]:
fname = '../configs/test.tsv'

def parse_config(fname):
    df = pd.read_csv(fname, sep='\t')
    df['path'] = df.fastq.str.rsplit('/', n=2, expand=True)[0]+'/'
    df['path2'] = df.fastq.str.rsplit('/', n=1, expand=True)[0]+'/'
    df['r2_fastq'] = df.fastq.str.replace('_R1_', '_R2_')
    return df

In [28]:
df = parse_config(fname)

In [29]:
df

,fastq,subpool,sample,path,path2,r2_fastq
0,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...,sub2,S1,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...
1,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...,sub2,S1,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...
2,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...,sub2,S1,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...
3,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...,sub2,S1,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...
4,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...,sub2,S1,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...
...,...,...,...,...,...,...
115,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...,sub16,S15,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1,/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Subli...
116,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...,sub16,S15,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...
117,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...,sub16,S15,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...
118,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...,sub16,S15,/dfs7/samlab/seyedam/IGVF/igvf_005,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2,/dfs7/samlab/seyedam/IGVF/igvf_005/nova2/Subli...


In [22]:
df.fastq.tolist()[:2]

['/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L001_R1_001.fastq.gz',
 '/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L002_R1_001.fastq.gz']

In [23]:
df.r2_fastq.tolist()[:2]

['/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L001_R2_001.fastq.gz',
 '/dfs7/samlab/seyedam/IGVF/igvf_005/nova1/Sublibrary_2_S1_L002_R2_001.fastq.gz']